In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import json

In [2]:
client = MongoClient('localhost', 27017)
db = client.db
comments = db.comments

In [3]:
commentdf = pd.DataFrame(comments.find())

In [4]:
def string_to_list(string):
    return json.loads(string)

def get_len(list):
    return len(list)

In [5]:
commentdf['stocks'] = commentdf['stocks'].apply(string_to_list)

In [6]:
commentdf['size'] = commentdf['stocks'].apply(get_len)

In [7]:
commentdf.loc[commentdf['size']!=0]

,_id,created_utc,body,score,sentiment,stocks,size
7,5fac60f84f1d40506940af77,1569935192,Any plays for today? Asides from hoping SPY is...,1,neutral,[SPY],1
8,5fac60f84f1d40506940af78,1569945346,"That would be amazing. \n\nCmon bear HFs, post...",3,bullish,[AAPL],1
10,5fac60f84f1d40506940af7a,1569941510,MSFT calls for tomorrow?,2,bullish,[MSFT],1
22,5fac60f84f1d40506940af87,1569940942,AAPL single handedly holding up the indexes,3,bullish,[AAPL],1
23,5fac60f84f1d40506940af88,1569959935,You heard it here first: MSFT calls are a bad ...,1,bearish,[MSFT],1
...,...,...,...,...,...,...,...
2392576,5faf404fa6460ea208f39e39,1601507530,$RGR,2,neutral,[RGR],1
2392581,5faf404fa6460ea208f39e3e,1601529664,"Cuz ""Sell the news"" - I don't make the stock m...",-2,bearish,[CUZ],1
2392612,5faf404fa6460ea208f39e5e,1601514509,Descending triangle on ZM. Puts will zoom,6,bearish,[ZM],1
2392615,5faf404fa6460ea208f39e61,1601546600,"I looked at MFA and thought it was ok, except ...",2,neutral,[MFA],1


In [8]:
stockdf = commentdf.loc[commentdf['size'] != 0]

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
s = stockdf['stocks']
mlb = MultiLabelBinarizer()
onehotdf = pd.DataFrame(mlb.fit_transform(s), columns = mlb.classes_, index = stockdf.index)

In [10]:
onehotdf

,A,AA,AACQ,AACQU,AAL,AAN,AAOI,AAP,AAPL,AAT,...,ZOM,ZS,ZSAN,ZTO,ZTS,ZUMZ,ZUO,ZVO,ZYNE,ZYXI
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2392576,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2392581,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2392612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2392615,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
connectionsdict = onehotdf.sum(axis = 1).to_dict()

In [12]:
connectionsdict = {k:v for k, v in connectionsdict.items() if v != 1}

In [21]:
onehotdf = onehotdf[onehotdf.index.isin(connectionsdict.keys())]

In [22]:
onehotdf

,A,AA,AACQ,AACQU,AAL,AAN,AAOI,AAP,AAPL,AAT,...,ZS,ZSAN,ZTO,ZTS,ZUMZ,ZUO,ZVO,ZYNE,ZYXI,BRK
52,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2392344,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2392413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2392510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2392539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
chorddict = onehotdf.sum(axis = 0).to_dict()

In [24]:
chorddict = {key:val for key, val in chorddict.items() if val > 100}

In [25]:
chorddict.keys()

dict_keys(['AAL', 'AAPL', 'ACB', 'ADBE', 'ALL', 'ALLY', 'AM', 'AMC', 'AMD', 'AMZN', 'AN', 'ANY', 'AONE', 'APHA', 'APT', 'AT', 'ATVI', 'AZN', 'BA', 'BABA', 'BAC', 'BB', 'BE', 'BEST', 'BIG', 'BILL', 'BOX', 'BP', 'BRO', 'BUD', 'BY', 'BYND', 'CAT', 'CCL', 'CEO', 'CGC', 'CHGG', 'CHWY', 'CLDR', 'CMG', 'COST', 'COTY', 'CRM', 'CRWD', 'CSCO', 'CSOD', 'CVS', 'CVX', 'CZR', 'DAL', 'DBX', 'DDOG', 'DELL', 'DGLY', 'DHT', 'DIS', 'DKNG', 'DNOW', 'DOCU', 'DOW', 'DPZ', 'DTE', 'EA', 'EBAY', 'ENPH', 'ET', 'ETSY', 'EURN', 'EVER', 'FB', 'FDS', 'FDX', 'FL', 'FMCI', 'FOR', 'FORD', 'FOX', 'FREE', 'FSLY', 'GDP', 'GE', 'GILD', 'GL', 'GLG', 'GM', 'GME', 'GOLD', 'GOOG', 'GOOGL', 'GRUB', 'GS', 'HAS', 'HD', 'HE', 'HOME', 'HOPE', 'HPQ', 'HTZ', 'HUYA', 'IBM', 'INO', 'INTC', 'IQ', 'IVR', 'JBLU', 'JD', 'JNJ', 'JP', 'JPM', 'KIM', 'KO', 'KODK', 'KR', 'LIFE', 'LIVE', 'LL', 'LMT', 'LOGI', 'LOW', 'LULU', 'LUV', 'LVGO', 'LVS', 'LYFT', 'LYV', 'MA', 'MAN', 'MCD', 'MGM', 'MRNA', 'MRO', 'MS', 'MSFT', 'MTB', 'MU', 'NAT', 'NCLH', 'N

In [30]:
nonMatrix = ['NaN', 'ACB', 'ALL', 'AM', 'AN', 'ANY', 'AONE', 'APT', 'AT', 'BB', 'BE', 'BEST', 'BIG', 'BILL', 'BOX', 'BRO', 'BY', 'CEO', 'DNOW', 'ET', 'EVER', 'FDS', 'FL', 'FOR', 'FREE', 'GDP', 'GL', 'HAS', 'HD', 'HE', 'HOME', 'HOPE', 'IQ', 'JP', 'KIM', 'LIFE', 'LIVE', 'LL', 'LOW', 'MAN', 'NEXT', 'NOW', 'OI', 'ONE', 'OUT', 'PLAN', 'PLAY', 'PM', 'PPT', 'PUMP', 'RE', 'REAL', 'RH', 'RTX', 'RUN', 'SAVE', 'SE', 'SGU', 'SO', 'SPXC', 'STAR          ', 'STAY', 'TDA', 'TEAM', 'TECH', 'TELL', 'TIMB$', 'TISI', 'TV', 'TWO', 'UI', 'TD', 'GLG']

In [27]:
take_or = lambda x, y: x if x >= y else y
onehotdf['BRK'] = onehotdf['BRK.A'].combine(onehotdf['BRK.B'], take_or)
onehotdf.drop(columns = ['BRK.A', 'BRK.B'], inplace = True)
onehotdf['GOOG'] = onehotdf['GOOG'].combine(onehotdf['GOOGL'], take_or)
onehotdf.drop(columns = ['GOOG', 'GOOGL'], inplace = True)

onehotdf.drop(columns = nonMatrix, inplace = True)


KeyError: 'BRK.A'

In [32]:
onehotdf.drop(columns = nonMatrix, inplace = True)

In [35]:
chorddict = onehotdf.sum(axis = 0).to_dict()

In [37]:
chorddict = {key:val for key, val in chorddict.items() if val > 100}

In [39]:
len(chorddict)

163

In [81]:
chorddf = onehotdf[chorddict.keys()]

In [82]:
chorddf = chorddf.T

In [84]:
testdict = chorddf.dot(chorddf.T).sum().sort_values().to_dict()

In [88]:
names = list(testdict.keys())[113:]

In [89]:
chorddf = onehotdf[names]

In [90]:
matrix = chorddf.corr()

In [91]:
matrix[matrix < 0] = 0
matrix[matrix == 1] = 0
matrix = matrix.values.tolist()

In [92]:
from chord import Chord


In [93]:
Chord(matrix, names).to_html()